# Exploration of OneMAP API service
This notebook is used for exploration on retreiving MRT/LRT stations location using OneMAP API search query via the public OneMAP API provided. Refer to https://www.onemap.gov.sg/apidocs/apidocs on the use of the search query API provided. 

Note: You would need to register an account to retreive the OneMAP API Key (inserted as request headers) for search query.

## Further information:
---
These OneMap APIs do not require a token to be used,
and there are no rate limits in place at this point:

* Search
* Basemaps
* Mini Map
* Advanced Mini Map
* Static Map

These OneMap APIs require a token to be renewed every three days.
There is also a 250 calls/min rate limit in place:
* Coordinate Converters
* Reverse Geocode
* Themes
* Routing
* Planning Area
* Population Query


In [1]:
#!/usr/bin/env python

import requests
import pandas as pd
import os,sys
PROJECT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, PROJECT_PATH)

from conf.api_key import ONE_MAP_KEY

# Helper functions

In [2]:
def get_stn_coordinates_info(searchVal: str, returnGeom : str ="Y", getAddrDetails: str = "N", onemap_url = "https://www.onemap.gov.sg/api/common/elastic/search?", api_key:str= ONE_MAP_KEY):

    searchVal = str(searchVal)
    # Space replacement for url construct
    searchVal = searchVal.strip().replace(" ","%20")

    # Construct search url
    onemap_search_url = onemap_url + f"searchVal={searchVal}&returnGeom={returnGeom}&getAddrDetails={getAddrDetails}"
    print(onemap_search_url)

    req_headers = {"User-agent": "qzq_test", 
                   "AccountKey": api_key, # OneMAP API key
                   "Content-Type": "application/json"
                   }
    station_coordinate_list = []
    res = requests.request("GET", onemap_search_url, headers=req_headers)
    # Check the status code before extending the number of posts
    if res.status_code == 200:
        print(f"Request successful with status code {res.status_code}")
        the_json = res.json()
        total_page = the_json["totalNumPages"]

        # Loop through each page of the paginated response to get all info
        for page in range(1, 1+total_page):
            onemap_search_url_page = f"{onemap_search_url}&pageNum={page}"
            print(f"Querying {onemap_search_url_page}")
            req_headers = {
                "User-agent": f"qzq_{page}", 
                "AccountKey": api_key, 
                "Content-Type": "application/json"
                }
            res = requests.request(
                "GET", onemap_search_url_page, headers=req_headers
            )
            if res.status_code == 200:
                print(f"Request successful with status code {res.status_code}")
                the_json = res.json()
                station_info = the_json["results"]
                station_coordinate_list.extend(station_info)
            else:
                print(f"Return unsuccessful with status code {res.status_code}")
                # Raise if HTTPError occured
                res.raise_for_status()
        return station_coordinate_list
    else:
        print(f"Return unsuccessful with status code {res.status_code}")
        # Raise if HTTPError occured
        res.raise_for_status()
    return None
    

In [3]:
mrt_search_response = get_stn_coordinates_info(searchVal="MRT STATION")
lrt_search_response = get_stn_coordinates_info(searchVal="LRT STATION")

https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N
Request successful with status code 200
Querying https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N&pageNum=1
Request successful with status code 200
Querying https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N&pageNum=2
Request successful with status code 200
Querying https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N&pageNum=3
Request successful with status code 200
Querying https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N&pageNum=4
Request successful with status code 200
Querying https://www.onemap.gov.sg/api/common/elastic/search?searchVal=MRT%20STATION&returnGeom=Y&getAddrDetails=N&pageNum=5
Request successful with status code 200
Querying https://www.onemap.g

In [4]:
print(len(mrt_search_response), len(lrt_search_response))
mrt_lrt_response_list = mrt_search_response + lrt_search_response
len(mrt_lrt_response_list)

954 44


998

In [5]:
# Quick check on contents
mrt_lrt_response_list

[{'SEARCHVAL': 'SOMERSET MRT STATION (NS23)',
  'X': '28641.8346576076',
  'Y': '31402.1245223526',
  'LATITUDE': '1.30026416739006',
  'LONGITUDE': '103.839085753124'},
 {'SEARCHVAL': 'LENTOR MRT STATION (TE5)',
  'X': '28269.897072956',
  'Y': '40827.7939434573',
  'LATITUDE': '1.38550657972169',
  'LONGITUDE': '103.835743809669'},
 {'SEARCHVAL': 'UPPER THOMSON MRT STATION (TE8)',
  'X': '27953.2465030747',
  'Y': '37390.003691975',
  'LATITUDE': '1.35441643365401',
  'LONGITUDE': '103.832898468504'},
 {'SEARCHVAL': 'CALDECOTT MRT STATION (TE9)',
  'X': '28761.5758000361',
  'Y': '35491.3431366347',
  'LATITUDE': '1.33724561271722',
  'LONGITUDE': '103.840161782075'},
 {'SEARCHVAL': 'CASHEW MRT STATION (DT2)',
  'X': '20334.5589111606',
  'Y': '39092.8585418891',
  'LATITUDE': '1.36981544925552',
  'LONGITUDE': '103.76443921414'},
 {'SEARCHVAL': 'MARINA BAY MRT STATION (NS27)',
  'X': '30367.9547207942',
  'Y': '28764.4959340338',
  'LATITUDE': '1.276410298755',
  'LONGITUDE': '103.8

Segregate extracted information into MRT/LRT station and station exits and save them into files.
- SVY21 coordinates are dropped in favor of Latitude or Longitude coordinates.

In [6]:
MRT_LRT_SEARCH_RESULTS_FILEPATH = os.path.join(PROJECT_PATH, "data", "MRT_LRT_location_exits.csv")
df = pd.DataFrame(mrt_lrt_response_list)
df.drop(columns=["X","Y"], inplace=True)

In [7]:
mrt_station_exit_df = df[df["SEARCHVAL"].str.contains("STATION EXIT")]
mrt_station_non_exit_df = df[df["SEARCHVAL"].str.contains("STATION \(")]


mrt_station_exit_df.rename(columns = {"SEARCHVAL": "StationExits", "LATITUDE": "Lat", "LONGITUDE": "Lon"}, inplace=True)

mrt_station_non_exit_df.rename(columns = {"SEARCHVAL": "StationName", "LATITUDE": "Lat", "LONGITUDE": "Lon"}, inplace=True)

mrt_station_non_exit_df["StationCode"] = mrt_station_non_exit_df["StationName"].map(lambda x: x.split(" ")[-1][1:-1])


C:\Users\quekz\AppData\Local\Temp\ipykernel_21240\459107947.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrt_station_exit_df.rename(columns = {"SEARCHVAL": "StationExits", "LATITUDE": "Lat", "LONGITUDE": "Lon"}, inplace=True)
C:\Users\quekz\AppData\Local\Temp\ipykernel_21240\459107947.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrt_station_non_exit_df.rename(columns = {"SEARCHVAL": "StationName", "LATITUDE": "Lat", "LONGITUDE": "Lon"}, inplace=True)
C:\Users\quekz\AppData\Local\Temp\ipykernel_21240\459107947.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [8]:
MRT_LRT_STN_FILEPATH = os.path.join(PROJECT_PATH, "data", "MRT_LRT_stn.csv")
MRT_LRT_STN_EXIT_FILEPATH = os.path.join(PROJECT_PATH, "data", "MRT_LRT_stn_exits.csv")
mrt_station_non_exit_df.to_csv(MRT_LRT_STN_FILEPATH, index=False)
mrt_station_exit_df.to_csv(MRT_LRT_STN_EXIT_FILEPATH, index=False)

In [ ]:
### Search for 